# Analyzing Fairness Metrics in Northpointe's COMPAS model 

<center>Human-Centric Artificial Intelligence Project </center> 
<br> Davi Nascimento 2016100642 <br> Gabriela Costa 2017254837

## Analyzing Racial Bias

É apresentado a previsão exagerada do modelo Compas

In [1]:
from truth_tables import PeekyReader, Person, table, is_race, count, vtable, hightable, vhightable
from csv import DictReader

people = []
with open("./data/cox-parsed.csv") as f:
    reader = PeekyReader(DictReader(f))
    try:
        while True:
            p = Person(reader)
            if p.valid:
                people.append(p)
    except StopIteration:
        pass

pop = list(filter(lambda i: ((i.recidivist == True and i.lifetime <= 730) or
                              i.lifetime > 730), list(filter(lambda x: x.score_valid, people))))
recid = list(filter(lambda i: i.recidivist == True and i.lifetime <= 730, pop))
rset = set(recid)
surv = [i for i in pop if i not in rset]

#### ALL DEFENDANTS

In [2]:
table(list(recid), list(surv))

         	Pred Yes  Pred No
Actual Yes	2681	1282	0.55
Actual No	1216	2035	0.45
Total: 7214.00
False positive rate: 0.32
False negative rate: 0.37
True positive rate: 0.63
True negative rate: 0.68
Accuracy: 0.65
Precision: 0.61
Pr(Y=1): 0.55


#### WHITE DEFENDANTS

In [3]:
is_white = is_race("Caucasian")
table(list(filter(is_white, recid)), list(filter(is_white, surv)))

         	Pred Yes  Pred No
Actual Yes	1139	349	0.61
Actual No	461	505	0.39
Total: 2454.00
False positive rate: 0.23
False negative rate: 0.48
True positive rate: 0.52
True negative rate: 0.77
Accuracy: 0.67
Precision: 0.59
Pr(Y=1): 0.61


#### BLACK DEFENDANTS

In [4]:
is_afam = is_race("African-American")
table(list(filter(is_afam, recid)), list(filter(is_afam, surv)))

         	Pred Yes  Pred No
Actual Yes	990	805	0.49
Actual No	532	1369	0.51
Total: 3696.00
False positive rate: 0.45
False negative rate: 0.28
True positive rate: 0.72
True negative rate: 0.55
Accuracy: 0.64
Precision: 0.63
Pr(Y=1): 0.49


Em geral, a taxa de falsos positivos é de 32% <br> Esta taxa é maior para afro-americanos, 45% <br>
E mais baixa para pessoas brancas-americanas, 23%.

In [5]:
45 / 23

1.9565217391304348

No COMPAS os arguidos afro-americanos têm cerca de 95% mais probabilidade de obter uma pontuação mais alta e de não cometer mais crimes do que os arguidos brancos após dois anos.

In [6]:
48 / 28

1.7142857142857142

O COMPAS classifica erradamente os reofensores brancos como sendo de baixo risco a 71.4% mais frequentemente do que os reofensores afro-americanos.

### Disparate Impact

Calculado como a relação entre a taxa de resultados favoráveis para o grupo não privilegiado e a do grupo privilegiado.

In [5]:
0.49 / 0.61

0.8032786885245902

O grupo privilegiado é beneficiado.

### Equal Opportunity Difference

Calculada como a diferença dos true positives rates entre os grupos não privilegiados e os privilegiados.

In [7]:
0.72 - 0.52

0.19999999999999996

O grupo privilegiado é beneficiado.

### Equalized odds difference

 Calculada como a diferença média da taxa de falsos positivos e da taxa de verdadeiros positivos entre grupos não privilegiados e privilegiados.

In [8]:
(0.44 - 0.23) / 2 + (0.72 - 0.52) / 2

0.20499999999999996

O grupo privilegiado é beneficiado.

### Statistical Parity Difference

Calculada como a diferença da taxa de resultados favoráveis recebidos pelo grupo não privilegiado em relação ao grupo privilegiado.

In [10]:
0.49 - 0.61

-0.12

O grupo privilegiado é beneficiado.